# Knowledge Graphs and Semantic Technologies -- RDFS tutorial

Here we'll learn the basics of RDFS (RDF Schema) and how to perform basic RDFS reasoning with rdflib (documentation [here](https://rdflib.readthedocs.io/en/stable/)) and owlrl (documentation [here](https://owl-rl.readthedocs.io/en/latest/)).

## Imports

owlrl is a library implementing basic RDFS and OWL reasoning on top of rdflib. We'll install and import its relevant symbols.

In [38]:
import sys
!{sys.executable} -m pip install rdflib owlrl

from rdflib import Graph, RDFS, RDF, URIRef
from owlrl import DeductiveClosure, RDFS_Semantics
from rdflib import Namespace

## Loading RDFS graphs

Your file `yourRDF.ttl` already contains a basic Knowledge Graph in RDF with some RDFS semantics

First, we are going to add some RDFS semantics, and inspect the graph as-is; this is also called the "asserted graph"

**Exercise 1** 
1. add additional triples using the RDFS semantics: have a look [here](https://www.w3.org/TR/rdf-schema/), and use domain and range, subPropertyOf, and Class, to say more about the instances in your graph
2. load yourRDF graph
3. print the classes in your graph
4. print the properties of a specific class in yourRDF graph
5. print all instances in yourRDF graph (all objects that have a type) 
6. explain what constitutes a vocabulary in RDF

In [39]:
#define graph
g = Graph()
g.parse("data/yourRDF.ttl")

#define example namespace
NS1 = Namespace("http://example.org")

#Add triples using store's add method.
g.add((NS1.Animalia, RDF.type, RDFS.Class))
g.add((NS1.Amphibia, RDF.type, RDFS.Class))
g.add((NS1.Mammalia, RDF.type, RDFS.Class))

g.add((NS1.eats, RDFS.domain, NS1.Animalia))
g.add((NS1.eats, RDFS.range, NS1.Animalia))

g.add((NS1.preysOn, RDFS.subPropertyOf, NS1.eats))

#print classes
print('classes:')
for c in g.subjects(RDF.type, RDFS.Class):
    print(c)

#print properties
print('properties of Animalia class:')
for p in g.subjects(RDFS.domain, NS1.Animalia):
    print(p)


#print all instances
print('all instances:')
for s, p, o in g.triples((None, RDF.type, None)):
    print(s, " is a ", o)

classes:
http://example.orgAnimalia
http://example.orgAmphibia
http://example.orgMammalia
properties of Animalia class:
http://example.orgeats
all instances:
https://example.org/whale  is a  https://example.org/Mammalia
https://example.org/crocodile  is a  https://example.org/Amphibia
http://example.orgAnimalia  is a  http://www.w3.org/2000/01/rdf-schema#Class
http://example.orgAmphibia  is a  http://www.w3.org/2000/01/rdf-schema#Class
http://example.orgMammalia  is a  http://www.w3.org/2000/01/rdf-schema#Class


A RDF vocabulary is a set of named terms, so URIs, that are used to describe data. RDF usually contains classes, properties and schema statements, that define how those terms should be used, for example rdfs.subClassOf. Such a vocabulary is often used with set to a namespace, as the 'NS1 = Namespace("http://example.org")'.

## RDFS inferencing

The inference engine in owlrl is triggered by `DeductiveClosure`, which computes the closure of the graph. This requires us to specify under which semantic regime we want to perform the inference (e.g. what kind of rules under the RDFS, OWL, etc. semantics we want the reasoner to produce derivations on). For RDFS semantics we use `RDFS_Semantics` as parameter. See extra options [here](https://owl-rl.readthedocs.io/en/latest/stubs/owlrl.html#module-owlrl)


**Exercise 2**
1. expand the graph through RDFS semantics inference
2. print how many triples the new graph has
3. print out the triples in your new graph and inspect them. 

In [41]:
#TIP:always look at linked documentation 
g_old = Graph()
g_old.parse('data/yourRDF.ttl')
g_new = Graph()
g_new.parse('data/yourRDF.ttl')
DeductiveClosure(RDFS_Semantics).expand(g_new)

print(f'The new graph as {len(g_new)} triples, whilst the old one has {len(g_old)}')

print('\nTriples: ')
for s, p, o in g_new.triples((None, None, None)):
    print(s, p, o)

The new graph as 22 triples, whilst the old one has 5

Triples: 
http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/2000/01/rdf-schema#subPropertyOf
https://example.org/crocodile http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://example.org/Animalia
https://example.org/crocodile http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
https://example.org/Amphibia http://www.w3.org/2000/01/rdf-schema#subClassOf https://example.org/Animalia
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/1999/02/22-rdf-syntax-ns#type
https://example.org/crocodile http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://example.org/Amphibia
https://example.org/whale https://example.org/eats https://example.org/crocodile
http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.

## The explicit (asserted) graph vs the implicit (derived) graph, and RDF entailment

Asserted triples are those that are explicitly stated, while derived or inferred triples are those that are implicitly stated through the semantics of RDFS. 

**Exercise 3**

1. Write here code to generate a graph that contains **RDFS derived triples only** from yourRDF Knowledge Graph, not the asserted ones. See a clue on rdflib graph algebra [here](https://rdflib.readthedocs.io/en/stable/merging.html)
2. have a look at the inferred graph. Based on the RDFS semantics, explain for each triple the rule that was used to generate it.
3. Explain the concept RDF entailment, and the types of entailment RDFS can produce


In [ ]:
derived_graph = Graph()
derived_graph = g_new - g_old
print(f'The derived graph has {len(derived_graph)} triples')

for s, p, o in derived_graph.triples((None, None, None)):
    print(s, p, o)


The asserted graph as 17 triples
http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/2000/01/rdf-schema#subPropertyOf
https://example.org/crocodile http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://example.org/Animalia
https://example.org/crocodile http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/1999/02/22-rdf-syntax-ns#Property
http://www.w3.org/2000/01/rdf-schema#subClassOf http://www.w3.org/2000/01/rdf-schema#subPropertyOf http://www.w3.org/2000/01/rdf-schema#subClassOf
https://example.org/Mammalia http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2000/01/rdf-schema#Resource
https://example

## Restaurant Assignment - Part 2


**Exercise 4**
1. load ingredients.rdf and recipes.rdf in one graph. The graph contains types of individuals and types of relationships between them. Print all the classes and properties in the combined graph with the namespace `ind` and the `wtm` namespace/vocabulary (`http://purl.org/heals/food/`). 

2. extend the `ind` vocabulary (`http://purl.org/heals/ingredient/`) by creating a hierarchy of ingredients (**hint: http://purl.org/heals/ingredient/CoconutMilk rdf:subClassOf http://purl.org/heals/ingredient/PlantMilk), and make these superclasses human readable by giving them labels**) 
3. do the same for the `wtm` vocabulary: add a hierarchy of recipes as well as a hierarchy of properties (**hint: http://purl.org/heals/food/hasCookingTemperature rdf:subPropertyOf ...) 
4. print the entailed triples as we did in the previous exercise
5. give three examples of how RDF semantics could aid the chefs in your restaurant 
    
6. which properties and classes could you add to the `wtm` and `ind` vocabularies to further describe your recipe and ingredient knowledge graph, aiding the chefs in your restaurant?  


In [ ]:
#In order to extend the vocabularies you have to come up with multiple examples that follows the given hierarchy

## HI ontology exploration

In your project, you will be working with a Hybrid Intelligence (HI) ontology. After acquanting yourself with its structure in the previous tutorial, you will now expand it yourself. Using the tools from the exercises above, perform the following actions:

1. Load the HI ontology from the data folder (hi_ontology.ttl) with RDFlib (don't forget to define a Namespace).
2. Create 3 new subclasses using the rdfs:subClassOf predicate.
3. Create at least one domain and one range restriction using rdfs.
4. Populate each class you've created with at least one new instance each (hint: use rdf:type).


